<a href="https://colab.research.google.com/github/IvanKatorgin/Recommender-systems/blob/main/%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B_%D0%B4%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Курс "Рекомендательные системы". Домашнее задание 3. Гибридные рекомендательные системы. Каторгин И.П.

Что нужно делать?

1. Датасет ml-latest.
2. Вспомнить подходы, которые мы разбирали.
3. Выбрать понравившийся подход к гибридным системам.
4. Написать свою.

In [1]:
# установим пакет Surprise
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357275 sha256=71beecfe94b091d62d50ac31075b6dddb60b0aad6dab615b9c5ec3ae71eabdf9
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [307]:
# импортируем библиотеки
import pandas as pd, numpy as np
from tqdm import tqdm_notebook
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cityblock, cosine, euclidean, hamming, jaccard
from surprise import KNNWithMeans, KNNBaseline, KNNBasic, Dataset, accuracy, Reader, SVD, SVDpp
from surprise.model_selection import cross_validate, train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

Для работы используем усеченную выборку из файла ml-latest-small.zip (size: 1 MB)

In [3]:
# загрузим датасет MovieLens с сайта https://grouplens.org/datasets/movielens/
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

--2024-12-03 15:26:07--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  2.46MB/s    in 0.4s    

2024-12-03 15:26:08 (2.46 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [308]:
# прочитаем файлы
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
links = pd.read_csv('ml-latest-small/links.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')

In [309]:
# объединим таблицы ratings и movies
movies_with_ratings = pd.merge(ratings, movies, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [310]:
# посмотрим на размер датасета (количество уникальных пользователей и фильмов)
print(f"фильмы: {len(movies_with_ratings['movieId'].unique())}.")
print(f"пользователи: {len(movies_with_ratings['userId'].unique())}.")

фильмы: 9724.
пользователи: 610.


## Гибридная рекомендательная система 1

Первый алгоритм

In [311]:
# датасет для Surprise
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [312]:
dataset.head()

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,1,Grumpier Old Men (1995),4.0
2,1,Heat (1995),4.0
3,1,Seven (a.k.a. Se7en) (1995),5.0
4,1,"Usual Suspects, The (1995)",5.0


In [313]:
# минимальное значение рейтинга
ratings.rating.min()

0.5

In [314]:
# максимальное значение рейтинга
ratings.rating.max()

5.0

In [315]:
# данные для создания модели
reader = Reader(rating_scale=(dataset['rating'].min(), dataset['rating'].max()))
data = Dataset.load_from_df(dataset, reader)

In [316]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

In [317]:
dataset['uid'].nunique(), dataset['iid'].nunique()

(610, 9719)

In [318]:
%%time
algo = SVD(n_factors=20, n_epochs=20, random_state=1)
algo.fit(trainset)

CPU times: user 789 ms, sys: 0 ns, total: 789 ms
Wall time: 795 ms


In [319]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8725


0.8724750940495861

In [320]:
algo.predict(uid=2.0, iid='Mortal Kombat (1995')

Prediction(uid=2.0, iid='Mortal Kombat (1995', r_ui=None, est=3.5189817180593166, details={'was_impossible': False})

In [321]:
current_user_id = 2.0
user_movies = movies_with_ratings[movies_with_ratings.userId == current_user_id].title.unique()

scores = []
titles = []

for movie in movies_with_ratings.title.unique():
    if movie in user_movies:
        continue

    scores.append(algo.predict(uid=current_user_id, iid=movie).est)
    titles.append(movie)

In [322]:
best_indices = np.argsort(scores)[-10:]
for i in reversed(best_indices):
    print(titles[i], scores[i])

Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) 4.4584014632823745
Star Wars: Episode IV - A New Hope (1977) 4.404709749278054
Lawrence of Arabia (1962) 4.386787014473195
Rear Window (1954) 4.384438738196943
Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001) 4.379797980406255
Casablanca (1942) 4.375516085230625
Godfather: Part II, The (1974) 4.372272957374455
Usual Suspects, The (1995) 4.365204129674977
Cool Hand Luke (1967) 4.348665960704037
Godfather, The (1972) 4.344178385456071


Второй алгоритм

In [323]:
def change_string(s):
    return s.replace(' ', '').replace('-', '')

movie_genres = [change_string(g) for g in movies.genres.values]
movie_genres[:10]

['Adventure|Animation|Children|Comedy|Fantasy',
 'Adventure|Children|Fantasy',
 'Comedy|Romance',
 'Comedy|Drama|Romance',
 'Comedy',
 'Action|Crime|Thriller',
 'Comedy|Romance',
 'Adventure|Children',
 'Action',
 'Action|Adventure|Thriller']

In [324]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(movie_genres)
X_train_tfidf

<9742x20 sparse matrix of type '<class 'numpy.float64'>'
	with 22084 stored elements in Compressed Sparse Row format>

In [325]:
neigh = NearestNeighbors(n_neighbors=20, metric='euclidean')
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='euclidean', n_neighbors=20)

In [326]:
test = change_string("Adventure|Comedy|Fantasy|Crime")

X_tfidf = tfidf.transform([test])

res = neigh.kneighbors(X_tfidf, return_distance=True)

In [327]:
X_tfidf

<1x20 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [328]:
res

(array([[0.42079615, 0.53300564, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608, 0.6188388 , 0.62682864, 0.62682864]]),
 array([[6774, 9096, 3576,  863, 2302, 2608, 7865, 3582, 8361, 3302, 5737,
         6723, 5636, 3376, 7496, 5627, 9717, 2206, 6133, 5832]]))

In [329]:
movies.iloc[res[1][0]]

,movieId,title,genres
6774,60074,Hancock (2008),Action|Adventure|Comedy|Crime|Fantasy
9096,143559,L.A. Slasher (2015),Comedy|Crime|Fantasy
3576,4899,Black Knight (2001),Adventure|Comedy|Fantasy
863,1136,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy
2302,3052,Dogma (1999),Adventure|Comedy|Fantasy
2608,3489,Hook (1991),Adventure|Comedy|Fantasy
7865,94015,Mirror Mirror (2012),Adventure|Comedy|Fantasy
3582,4911,Jabberwocky (1977),Adventure|Comedy|Fantasy
8361,109042,Knights of Badassdom (2013),Adventure|Comedy|Fantasy
3302,4467,"Adventures of Baron Munchausen, The (1988)",Adventure|Comedy|Fantasy


In [330]:
title_genres = {}

for index, row in tqdm(movies.iterrows()):
    title_genres[row.title] = row.genres

title_genres

0it [00:00, ?it/s]

{'Toy Story (1995)': 'Adventure|Animation|Children|Comedy|Fantasy',
 'Jumanji (1995)': 'Adventure|Children|Fantasy',
 'Grumpier Old Men (1995)': 'Comedy|Romance',
 'Waiting to Exhale (1995)': 'Comedy|Drama|Romance',
 'Father of the Bride Part II (1995)': 'Comedy',
 'Heat (1995)': 'Action|Crime|Thriller',
 'Sabrina (1995)': 'Comedy|Romance',
 'Tom and Huck (1995)': 'Adventure|Children',
 'Sudden Death (1995)': 'Action',
 'GoldenEye (1995)': 'Action|Adventure|Thriller',
 'American President, The (1995)': 'Comedy|Drama|Romance',
 'Dracula: Dead and Loving It (1995)': 'Comedy|Horror',
 'Balto (1995)': 'Adventure|Animation|Children',
 'Nixon (1995)': 'Drama',
 'Cutthroat Island (1995)': 'Action|Adventure|Romance',
 'Casino (1995)': 'Crime|Drama',
 'Sense and Sensibility (1995)': 'Drama|Romance',
 'Four Rooms (1995)': 'Comedy',
 'Ace Ventura: When Nature Calls (1995)': 'Comedy',
 'Money Train (1995)': 'Action|Comedy|Crime|Drama|Thriller',
 'Get Shorty (1995)': 'Comedy|Crime|Thriller',
 'Copy

Соединяем вместе

In [331]:
movies_with_ratings[movies_with_ratings.userId == 2].sort_values('timestamp').title.values

array(['Shawshank Redemption, The (1994)', 'Inception (2010)',
       'The Jinx: The Life and Deaths of Robert Durst (2015)',
       'Ex Machina (2015)', 'Django Unchained (2012)', 'Whiplash (2014)',
       'Gladiator (2000)', 'Dark Knight Rises, The (2012)',
       'Shutter Island (2010)', 'Girl with the Dragon Tattoo, The (2011)',
       'Exit Through the Gift Shop (2010)', 'Kill Bill: Vol. 1 (2003)',
       'Collateral (2004)', 'Wolf of Wall Street, The (2013)',
       'Zombieland (2009)', 'Step Brothers (2008)',
       'Talladega Nights: The Ballad of Ricky Bobby (2006)',
       'Tommy Boy (1995)', 'Departed, The (2006)',
       'Dark Knight, The (2008)', 'Interstellar (2014)',
       'Inglourious Basterds (2009)', 'Louis C.K.: Hilarious (2010)',
       'Inside Job (2010)', 'Warrior (2011)', 'Good Will Hunting (1997)',
       'Mad Max: Fury Road (2015)', 'The Drop (2014)', 'Town, The (2010)'],
      dtype=object)

In [332]:
res[1][0]

array([6774, 9096, 3576,  863, 2302, 2608, 7865, 3582, 8361, 3302, 5737,
       6723, 5636, 3376, 7496, 5627, 9717, 2206, 6133, 5832])

In [333]:
movies[movies['title'] == 'Town, The (2010)'].movieId.iloc[0]

80489

In [334]:
def recommend_for_user(user_id):
    user_movies = movies_with_ratings[movies_with_ratings.userId == user_id].sort_values('timestamp').title.values

    # первый
    last_user_movie = user_movies[-1]

    movie_genres = change_string(title_genres[last_user_movie])

    predict = tfidf.transform([movie_genres])
    res = neigh.kneighbors(X_tfidf, return_distance=True)

    movies_to_score = movies.loc[res[1][0]].title.values[1:]

    # второй
    scores = []
    titles = []

    for movie in movies_to_score:
        if movie in user_movies:
            continue

        scores.append(algo.predict(uid=user_id, iid=movie).est)
        titles.append(movie)

    best_indices = np.argsort(scores)[-10:]
    for i in reversed(best_indices):
        print(titles[i], scores[i])

In [335]:
movies_with_ratings[movies_with_ratings.userId == 2.0].sort_values('rating').tail()

,userId,movieId,rating,timestamp,title,genres
248,2,80906,5.0,1445715172,Inside Job (2010),Documentary
241,2,60756,5.0,1445714980,Step Brothers (2008),Comedy
250,2,89774,5.0,1445715189,Warrior (2011),Drama
254,2,106782,5.0,1445714966,"Wolf of Wall Street, The (2013)",Comedy|Crime|Drama
260,2,131724,5.0,1445714851,The Jinx: The Life and Deaths of Robert Durst ...,Documentary


In [336]:
recommend_for_user(2)

Monty Python and the Holy Grail (1975) 4.257347775116619
Life Aquatic with Steve Zissou, The (2004) 3.8286976628899003
Dogma (1999) 3.738399672508874
Adventures of Baron Munchausen, The (1988) 3.6998554294950154
Time of the Gypsies (Dom za vesanje) (1989) 3.6940464113217284
Asterix & Obelix: Mission Cleopatra (Astérix & Obélix: Mission Cléopâtre) (2002) 3.639220385339219
Hook (1991) 3.5887448417160117
Jabberwocky (1977) 3.5405743291990044
L.A. Slasher (2015) 3.5189817180593166
The Man Who Killed Don Quixote (2018) 3.5189817180593166


## Гибридная рекомендательная система 2. Моя модель

In [337]:
# объединим таблицы с рейтингами и названиями фильмов

movies_ratings = movies.merge( ratings, on='movieId').reset_index(drop=True)
movies_ratings.dropna(inplace=True)

movies_ratings.head(5)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [338]:
# датасет для Surprise

dataset = pd.DataFrame( {
    'uid'   : movies_ratings['userId'],
    'iid'   : movies_ratings['title'],
    'rating' : movies_ratings['rating']
})

In [339]:
# данные для создания модели
reader = Reader(rating_scale=(dataset['rating'].min(), dataset['rating'].max()))
data = Dataset.load_from_df(dataset, reader)

In [340]:
# модель KNNBaseline
# модель рекомендации на рейтингах фильма

trainset, testset = train_test_split(data, test_size=0.15, random_state=42)

model_KNN = KNNBaseline(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})

model_KNN.fit(trainset)

test_pred = model_KNN.test(testset)
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8461


0.8460980885360158

In [341]:
# model_KNN
# рекомендации

current_user_id = 10.0
user_movies = movies_ratings[movies_ratings.userId == current_user_id].title.unique()

scores = []
titles = []

for movie in movies_ratings.title.unique():
    if movie in user_movies:
        continue

    scores.append(model_KNN.predict(uid=current_user_id, iid=movie).est)
    titles.append(movie)


sorted(scores)[-10:]

[4.284138477610336,
 4.358557940675762,
 4.374199084838914,
 4.378197085843423,
 4.444719880495544,
 4.539301551458315,
 4.585987287038421,
 4.661495038567163,
 4.744808374161465,
 4.9181700035577]

In [342]:
#  модель рекомендации на жанрах фильма
#  NearestNeighbors подход (лгоритма ближайших соседей на жанрах)

# функци парсинга жанров фильма

def change_string(g):
    return g.replace(' ', '').replace('-', '').replace('|', ' ')

movie_genres = [change_string(x) for x in movies.genres.values]

# Tfidf преобразование текста в вектор
count_vect = CountVectorizer()
x_train = count_vect.fit_transform(movie_genres)

tfidf_transformer = TfidfTransformer()
x_tfidf = tfidf_transformer.fit_transform(x_train)

model_neig = NearestNeighbors(n_neighbors=50, n_jobs=-1, metric='euclidean')
model_neig.fit(x_tfidf);

In [343]:
title_genres = {}

for index, row in tqdm_notebook(movies.iterrows()):
    title_genres[row.title] = row.genres

0it [00:00, ?it/s]

In [344]:
# функция рекомендации фильма

def recommend_for_user(user_id):

    # список фильмов для пользователя
    user_movies = movies_ratings [movies_ratings ["userId"] == user_id].sort_values('timestamp').title.values

    # последний просмотренный пользователем фильм
    last_user_movie = user_movies[-1]

    # жанры фильма преобразуем в векток TF-IDF
    movi_genres = change_string(title_genres[last_user_movie])

    #  получаем предсказание по жанру последнего фильма

    predict = count_vect.transform([movi_genres])
    x_tfidf2 = tfidf_transformer.transform(predict)

    res = model_neig.kneighbors(x_tfidf, return_distance=True)

    movies_to_score = movies.iloc[res[1][0]].title.values

    # для отобранных фильмов, предсказываем оценку на скрытых факторах
    scores = []
    title = []

    for movie in movies_to_score:
        if movie in user_movies:
            continue
        scores.append(model_KNN.predict(uid=user_id, iid=movie).est)
        title.append(movie)

    #  формируем датафрейм с рекомендациями
    best_indexes = np.argsort(scores)[-10:]
    mov = []
    scor = []

    for i in reversed(best_indexes):

        mov.append(title[i])
        scor.append(scores[i])

    return pd.DataFrame({"Movie": mov, "Rating": scor})

In [345]:
#  рекомендации для пользователя
user_id = 45
print(f"Рекомендации для пользователя с ID = {user_id}.")
recommend_for_user(user_id)

Рекомендации для пользователя с ID = 45.


,Movie,Rating
0,Ponyo (Gake no ue no Ponyo) (2008),4.542196
1,Kiki's Delivery Service (Majo no takkyûbin) (1...,4.418659
2,Partly Cloudy (2009),4.404881
3,Inside Out (2015),4.373526
4,The Lego Movie (2014),4.341374
5,"Twelve Tasks of Asterix, The (Les douze travau...",4.290709
6,Kubo and the Two Strings (2016),4.284643
7,Cloudy with a Chance of Meatballs 2 (2013),4.266635
8,Moana (2016),4.155809
9,Watership Down (1978),4.148004


## Гибридная рекомендательная система 3. Модель LightFM

In [35]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808331 sha256=704ecc16a79f6e851b9dc89d17a67f1f405bb89646ad9327609cf1ff0e0ef85b
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [346]:
from lightfm import LightFM

In [347]:
movies_with_ratings

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Split (2017),Drama|Horror|Thriller
100832,610,168248,5.0,1493850091,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,610,168250,5.0,1494273047,Get Out (2017),Horror
100834,610,168252,5.0,1493846352,Logan (2017),Action|Sci-Fi


In [348]:
interactions = movies_with_ratings.groupby(['userId', 'title'])['rating'].count().unstack().fillna(0)
interactions.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [349]:
from scipy import sparse

interaction_sparse = sparse.csr_matrix(interactions)
interaction_sparse

<610x9719 sparse matrix of type '<class 'numpy.float64'>'
	with 100832 stored elements in Compressed Sparse Row format>

In [350]:
model = LightFM(no_components=30, random_state=10, loss='logistic')
model.fit(interaction_sparse, epochs=10)

In [351]:
n_items = interactions.shape[1]
user_id = 2
scores = model.predict(user_id, np.arange(n_items))
scores

array([1.3403213, 1.3756119, 1.4453913, ..., 1.4753232, 1.9412823,
       1.3893595], dtype=float32)

In [352]:
np.arange(n_items)

array([   0,    1,    2, ..., 9716, 9717, 9718])

In [353]:
len(scores)

9719

In [354]:
scores = pd.Series(scores)
scores.index = interactions.columns
scores

,0
title,
'71 (2014),1.340321
'Hellboy': The Seeds of Creation (2004),1.375612
'Round Midnight (1986),1.445391
'Salem's Lot (2004),1.419254
'Til There Was You (1997),1.476702
...,...
eXistenZ (1999),1.898718
xXx (2002),1.886755
xXx: State of the Union (2005),1.475323


In [355]:
user_row = interactions.loc[user_id]
known_items = user_row[user_row != 0].index
len(known_items)

29

In [356]:
user_row

,2
title,
'71 (2014),0.0
'Hellboy': The Seeds of Creation (2004),0.0
'Round Midnight (1986),0.0
'Salem's Lot (2004),0.0
'Til There Was You (1997),0.0
...,...
eXistenZ (1999),0.0
xXx (2002),0.0
xXx: State of the Union (2005),0.0


In [357]:
unknown_items = list(set(interactions.columns) - set(known_items))
scores = scores[unknown_items].sort_values(ascending=False)
scores

,0
title,
Forrest Gump (1994),3.645114
Pulp Fiction (1994),3.596703
"Silence of the Lambs, The (1991)",3.523236
"Matrix, The (1999)",3.480274
Braveheart (1995),3.417051
...,...
Collision Course (1989),1.321920
Tightrope (1984),1.321741
Robot Jox (1990),1.321631


In [358]:
return_score_list = scores[:5]
return_score_list

,0
title,
Forrest Gump (1994),3.645114
Pulp Fiction (1994),3.596703
"Silence of the Lambs, The (1991)",3.523236
"Matrix, The (1999)",3.480274
Braveheart (1995),3.417051


## Дополнительная модель 1

In [359]:
from lightfm.datasets import fetch_movielens

movielens = fetch_movielens()

In [360]:
movielens

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 90570 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 9430 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1'),
 'item_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1')}

In [361]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse._csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [362]:
train = movielens['train']
test = movielens['test']

In [363]:
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM()
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.43, test 0.13.


In [364]:
model = LightFM(learning_rate=0.05, loss='warp')

model.fit_partial(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.59, test 0.21.


In [365]:
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()
                          [user_id].indices]

        scores = model.predict(user_id, np.arange(n_items))

        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("Known positives:")

        for x in known_positives[:3]:
            print(x)

        print("Recommended:")

        for x in top_items[:3]:
            print(x)

In [366]:
sample_recommendation(model, movielens, [18, 50, 340])

User 18
Known positives:
B
G
I
Recommended:
E
F
C
User 50
Known positives:
S
C
M
Recommended:
R
S
E
User 340
Known positives:
G
H
1
Recommended:
A
D
S


Почему-то выдает только первые буквы, а не названия фильма целиком. Непонятно почему???

## Дополнительная модель 2. Content_based model

In [367]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [368]:
def change_string(s):
    return s.replace(' ', '').replace('-', '')

movie_genres = [change_string(g) for g in movies.genres.values]
movie_genres[:10]

['Adventure|Animation|Children|Comedy|Fantasy',
 'Adventure|Children|Fantasy',
 'Comedy|Romance',
 'Comedy|Drama|Romance',
 'Comedy',
 'Action|Crime|Thriller',
 'Comedy|Romance',
 'Adventure|Children',
 'Action',
 'Action|Adventure|Thriller']

In [369]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(movie_genres)

features = tfidf.get_feature_names_out()
tfidf_movie = pd.DataFrame(X_train_tfidf.toarray(), columns=features)
tfidf_movie.head()

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,0.0,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [370]:
tfidf_movie['movieId'] = movies['movieId']
tfidf_movie.head()

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,...,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western,movieId
0,0.0,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1
1,0.0,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2
2,0.0,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0,3
3,0.0,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0,4
4,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,5


In [371]:
from lightfm.data import Dataset

ds = Dataset()
ds

In [372]:
ds.fit(users=movies_with_ratings['userId'].unique(),
       items=movies['movieId'],
       item_features=features)

In [373]:
def transform_features(features, id_name):
    """
    Transforms features data frame to the format build_user_features / build_item_features needs it.

    Returns iterable of (id, [list of feature names that has value]).
    """
    transformed_features = []
    for row in features.to_dict(orient="records"):
        id_value = row[id_name]
        del row[id_name]
        feature_names = {key: value for key, value in row.items() if value != 0}
        transformed_features.append(
            (id_value, feature_names)
        )
    return transformed_features

In [374]:
transform_features(tfidf_movie, "movieId")

[(1,
  {'adventure': 0.41684567364693936,
   'animation': 0.5162254711770092,
   'children': 0.5048454681396087,
   'comedy': 0.26758647689140014,
   'fantasy': 0.482990142708577}),
 (2,
  {'adventure': 0.5123612074824269,
   'children': 0.6205251727456431,
   'fantasy': 0.5936619434123594}),
 (3, {'comedy': 0.5709154064399099, 'romance': 0.8210088907493954}),
 (4,
  {'comedy': 0.5050154397005037,
   'drama': 0.46640480307738325,
   'romance': 0.726240982959826}),
 (5, {'comedy': 1.0}),
 (6,
  {'action': 0.5493281743985543,
   'crime': 0.6359470441562757,
   'thriller': 0.5420423542868653}),
 (7, {'comedy': 0.5709154064399099, 'romance': 0.8210088907493954}),
 (8, {'adventure': 0.6366993258087036, 'children': 0.7711121633813997}),
 (9, {'action': 1.0}),
 (10,
  {'action': 0.5530653284926609,
   'adventure': 0.6295217016667962,
   'thriller': 0.5457299419583338}),
 (11,
  {'comedy': 0.5050154397005037,
   'drama': 0.46640480307738325,
   'romance': 0.726240982959826}),
 (12, {'comedy': 

In [375]:
item_features_matrix = ds.build_item_features(transform_features(tfidf_movie, "movieId"))
item_features_matrix

<9742x9762 sparse matrix of type '<class 'numpy.float32'>'
	with 31826 stored elements in Compressed Sparse Row format>

In [376]:
data = []
for i, row in movies_with_ratings.iterrows():
    data.append((row['userId'], row['movieId']))

data[:10]

[(1, 1),
 (5, 1),
 (7, 1),
 (15, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (21, 1),
 (27, 1),
 (31, 1)]

In [377]:
interaction_matrix = ds.build_interactions(data=data)[0]
interaction_matrix

<610x9742 sparse matrix of type '<class 'numpy.int32'>'
	with 100836 stored elements in COOrdinate format>

In [378]:
from scipy import sparse

model = LightFM(no_components=30, random_state=10, loss='bpr')
model.fit(interaction_matrix, epochs=10, item_features=item_features_matrix)

In [379]:
n_items = interaction_matrix.shape[1]
user_id = 2
scores = model.predict(user_id, np.arange(n_items))
scores

array([-2.0344148, -2.6314387, -3.5380545, ..., -3.4019089, -3.5515428,
       -3.3523276], dtype=float32)

In [380]:
len(scores)

9742

In [381]:
scores = pd.Series(scores)
scores.index = movies['title']
scores.sort_values(ascending=False)

,0
title,
Forrest Gump (1994),0.158636
"Lord of the Rings: The Return of the King, The (2003)",-0.870440
"Matrix, The (1999)",-0.938254
Pulp Fiction (1994),-0.949107
"Shawshank Redemption, The (1994)",-0.997824
...,...
Run Lola Run (Lola rennt) (1998),-4.200052
Heathers (1989),-4.201553
Under Siege 2: Dark Territory (1995),-4.225579


Выводы:

В рамках домашнего задания были построены 3 различные гибридные модели системы рекомендаций, а также 2 дополнительные модели. Каких-то личных предпочтений выбора определенной модели у меня не сформировалось. Каждая модель по-своему хороша и все модели в дальнейшем можно использовать.